# Svensson (1994)

We can use the genetic algorithm to optimize a simple equation, such as the one below.

$$ r(t) = \beta_1 + \beta_2\frac{1-e^{-\lambda_1 t}}{\lambda_1 t} 
        + \beta_3 \left(\frac{1-e^{-\lambda_1 t}}{\lambda_1 t}-e^{-\lambda_1 t}\right)
        + \beta_4 \left(\frac{1-e^{-\lambda_2 t}}{\lambda_2 t}-e^{-\lambda_2 t}\right)$$

### Importing the data

In [300]:
from scipy.optimize import fmin
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from workadays import workdays as wd

path = r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\Brasil\ETTJ\Base_LTN.xlsx"
path2 = r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\Brasil\ETTJ\Parametros.xlsx"
df = pd.read_excel(path)
parameters = pd.read_excel(path2)

In [301]:
parameters.set_index("Data referência", inplace=True)
df["Maturity"] = df.apply(lambda row: wd.networkdays(row["DATA_REFERENCIA"], row["DATA_VENCIMENTO"]), axis=1)/252
df.set_index("DATA_REFERENCIA", inplace=True)

### Generating individuals

In [318]:
N =1200    ### Number of individuals 
p = int(2/3*N)      ## Proportion of individuals generated by method 1
var = 0.5

### 1 - Initial Values of the results of the estimation of the previous day

ref_date_str = "2023-07-27"
ref_date = datetime.strptime(ref_date_str, "%Y-%m-%d")
previous_date = ref_date - timedelta(days=1)
previous_parameters = parameters.loc[previous_date][1:]
beta_star = np.array([previous_parameters.replace(previous_parameters[1],previous_parameters[0])])
e_previous = beta_star.T*np.random.normal(0, var, size=(6, p))
beta_previous = beta_star.T + e_previous

### 2 - Approximation that takes into account the observed yield to maturity (ytm)

data_estimada = df.loc[ref_date_str]
data_estimada.sort_values("Maturity")
b1 = data_estimada["EXPECTATIVA"][0]/100
b2 = data_estimada["EXPECTATIVA"][-1]/100 - data_estimada["EXPECTATIVA"][0]/100
b3 = 0
b4 = 0
lbda1 = (data_estimada["Maturity"][-1]-data_estimada["Maturity"][0])/2
lbda2 = lbda1
beta_star = np.array([b1, b2, b3 , b4, lbda1, lbda2])[:, np.newaxis]
e_approx = beta_star*np.random.normal(0, var, size=(6, N-p))
beta_approx = beta_star + e_approx ## ajustar colunas

individuals = np.concatenate((beta_previous.T, beta_approx.T), axis=0)
individuals.shape

### incluir restricoes nos betas e lambdas

(1200, 6)

### Selection

In [388]:
### add duration and coupons

for row in individuals:
    def myval(c):
        df['NSS'] =(c[0])+(c[1]*((1-np.exp(-df['Maturity']*c[4]))/(df['Maturity']*c[4])))+(c[2]*((((1-np.exp(-df['Maturity']*c[4]))/(df['Maturity']*c[4])))-(np.exp(-df['Maturity']*c[4]))))+(c[3]*((((1-np.exp(-df['Maturity']*c[5]))/(df['Maturity']*c[5])))-(np.exp(-df['Maturity']*c[5]))))
        df['Calculated_price'] = 1000 / (1 + df['NSS']) ** df['Maturity'] 
        df['Residual'] =  (df['PU'] - df['Calculated_price'])**2
        val = np.sum(df['Residual'])
        return(val)
    
    val = myval(row)
    values.append((val, row)) 
sol = pd.DataFrame(values, columns=['Calculated Value', "Parameters"])

selection = sol.sort_values('Calculated Value')[0:40]

In [390]:
selection.head()

,Calculated Value,Parameters
476,136.802466,"[0.11876233876266598, -0.008343678688489814, 0..."
4076,136.802466,"[0.11876233876266598, -0.008343678688489814, 0..."
2876,136.802466,"[0.11876233876266598, -0.008343678688489814, 0..."
1676,136.802466,"[0.11876233876266598, -0.008343678688489814, 0..."
263,170.045710,"[0.08950800403278611, 0.04862749458094612, 0.0..."


### Cross-Over and Mutation